In [ ]:
!nvidia-smi 


/bin/bash: nvidia-smi: command not found


In [ ]:
!bash get_data.sh

In [1]:
!pip install -r requirements.txt


In [2]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import ViTModel, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup
from PIL import Image
import numpy as np
from model import GPT2DecoderWithImageFeatures
from torchvision import transforms

vocabulary = ', { } small-title text quadruple row btn-inactive btn-orange btn-green btn-red double <START> header btn-active <END> single <UNK> <PAD>'.split()
special_tokens_dict = {'additional_special_tokens': vocabulary}

img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Load and preprocess data
class Pix2CodeDataset(Dataset):
    def __init__(self, data_path, img_transform, dsl_transform, mode="train", split_ratio=0.8):
        self.data_path = data_path
        self.img_transform = img_transform
        self.dsl_transform = dsl_transform
        self.mode = mode
        self.split_ratio = split_ratio
        self.data = self.load_data()

    def load_data(self):
        data = []
        for root, _, files in os.walk(self.data_path):
            for file in files:
                if file.endswith(".png"):
                    img_path = os.path.join(root, file)
                    dsl_path = os.path.splitext(img_path)[0] + ".gui"
                    data.append((img_path, dsl_path))

        split_index = int(len(data) * self.split_ratio)
        if self.mode == "train":
            return data[:split_index]
        elif self.mode == "val":
            return data[split_index:]
        else:
            raise ValueError("Invalid mode. Use 'train' or 'val'.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, dsl_path = self.data[idx]
        img_rgb = Image.open(img_path)
        img_grey = img_rgb.convert("L")
        img_adapted = img_grey.point(lambda x: 255 if x > 128 else 0)
        img_stacked = np.stack((img_adapted, img_adapted, img_adapted), axis=-1)
        img_stacked_pil = Image.fromarray(np.uint8(img_stacked), mode='RGB')

        with open(dsl_path, "r") as f:
            dsl_code = f.read()

        img_tensor = self.img_transform(img_stacked_pil)
        dsl_tokens = self.dsl_transform('<START>\n' + dsl_code + '\n<END>')
        dsl_tensor = torch.LongTensor(dsl_tokens)

        return img_tensor, dsl_tensor

# Initialize ViT model, dataset, and data loader
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224').base_model

# Replace the dsl_transform with the tokenizer.encode method
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens(special_tokens_dict)
dsl_transform = tokenizer.encode

# Create train and validation data loaders
data_path = "data/all_data/data"

train_dataset = Pix2CodeDataset(data_path, img_transform, dsl_transform, mode="train")
val_dataset = Pix2CodeDataset(data_path, img_transform, dsl_transform, mode="val")

def pad_collate_fn(batch):
    imgs, dsls = zip(*batch)

    # Pad DSL sequences
    max_len = max([len(dsl) for dsl in dsls])
    end_token = tokenizer.encode('<PAD>')[0]
    padded_dsls = []
    for dsl in dsls:
        padded_dsls.append(torch.cat([dsl, torch.full((max_len - len(dsl),), end_token, dtype=torch.long        )]))

    # Stack padded DSL sequences and images
    img_tensor = torch.stack(imgs)
    dsl_tensor = torch.stack(padded_dsls)

    return img_tensor, dsl_tensor

batch_size = 32
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate_fn)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate_fn)

# Define hyperparameters
input_size = 768
num_layers = 6
epochs = 1000
learning_rate = 0.0001

# Initialize the decoder, loss function, and optimizer
decoder = GPT2DecoderWithImageFeatures(input_size)
decoder.gpt.resize_token_embeddings(len(tokenizer))  # Update the GPT2 model with the new tokenizer
decoder.load_state_dict(torch.load("best_decoder.pth"))
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(decoder.parameters(), lr=learning_rate)

# Initialize the learning rate scheduler with warm-up
num_warmup_steps = 500
num_training_steps = epochs * len(train_data_loader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# Initialize variables to track the best validation loss and epoch
best_val_loss = float("inf")
best_epoch = 0

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_model.to(device)
decoder.to(device)

vit_model.eval()

for epoch in range(epochs):
    for i, (img_tensor, dsl_tensor) in enumerate(train_data_loader):
        loss = 0
        decoder.train()
        img_tensor = img_tensor.to(device)
        dsl_tensor = dsl_tensor.to(device)

        # Extract image features using ViT model
        with torch.no_grad():
            image_features = vit_model(img_tensor).last_hidden_state[:, 0, :]

        # Training loop for the GPT2DecoderWithImageFeatures
        input_tokens = dsl_tensor[:, :-1]
        target_tokens = dsl_tensor[:, 1:]
       
        output = decoder(input_tokens, image_features)

        # print(output.shape)
        # print(target_tokens.shape)

        loss = criterion(output.permute(0, 2, 1), target_tokens)

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(decoder.parameters(), max_norm=1.0)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        if i % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Step {i}/{len(train_data_loader)}, Loss: {loss.item()}")

    # Evaluate the model on the validation set
    val_loss = 0
    decoder.eval()
    with torch.no_grad():
        for i, (img_tensor, dsl_tensor) in enumerate(val_data_loader):
            img_tensor = img_tensor.to(device)
            dsl_tensor = dsl_tensor.to(device)

            image_features = vit_model(img_tensor).last_hidden_state[:, 0, :]

            input_tokens = dsl_tensor[:, :-1]
            target_tokens = dsl_tensor[:, 1:]     

            output = decoder(input_tokens, image_features)

            val_loss += criterion(output.permute(0, 2, 1), target_tokens).item()

    val_loss /= len(val_data_loader)
    print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {val_loss}")
    # Save the best model weights
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch = epoch
        torch.save(decoder.state_dict(), "best_decoder.pth")

print(f"Best model weights saved from epoch {best_epoch+1} with validation loss {best_val_loss}")

   



d:\work\image2dsl\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bi

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
import torch
from transformers import GPT2Tokenizer
from PIL import Image
from model import GPT2DecoderWithImageFeatures
from transformers import ViTModel

vocabulary = ', { } small-title text quadruple row btn-inactive btn-orange btn-green btn-red double <START> header btn-active <END> single <UNK> <PAD>'.split()
special_tokens_dict = {'additional_special_tokens': vocabulary}

def generate_code(image_path, tokenizer, vit_model, decoder, max_length=512):
    # Load and preprocess the image
    img_rgb = Image.open(image_path)
    img_grey = img_rgb.convert("L")
    img_adapted = img_grey.point(lambda x: 255 if x > 128 else 0)
    img_stacked = np.stack((img_adapted, img_adapted, img_adapted), axis=-1)
    img_stacked_pil = Image.fromarray(np.uint8(img_stacked), mode='RGB')
    img_tensor = img_transform(img_stacked_pil)
    img_tensor = img_tensor.unsqueeze(0).to(device)

    # Extract image features using ViT model
    with torch.no_grad():
        image_features = vit_model(img_tensor).last_hidden_state[:, 0, :]

    # Prepare the initial input for the decoder
    input_ids = tokenizer.encode('<START>', return_tensors='pt').to(device)

    # Generate code using the decoder
    decoder.eval()
    generated_code = []
    for _ in range(max_length):
        with torch.no_grad():
            output = decoder(input_ids, image_features)
        
        next_token_id = torch.argmax(output, dim=-1)[:, -1]
        input_ids = torch.cat([input_ids, next_token_id.unsqueeze(-1)], dim=-1)
        
        if next_token_id.item() == tokenizer.encode('<END>')[0]:
            break

        generated_code.append(tokenizer.decode(next_token_id))

    return ''.join(generated_code)

# Load the tokenizer, ViT model, and decoder
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens_dict = {'additional_special_tokens': vocabulary}
tokenizer.add_special_tokens(special_tokens_dict)
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224').base_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_model.to(device)

decoder = GPT2DecoderWithImageFeatures(input_size=768)
decoder.gpt.resize_token_embeddings(len(tokenizer))  # Update the GPT2 model with the new tokenizer

decoder.load_state_dict(torch.load("best_decoder.pth"))
decoder.to(device)

# Perform inference on a test image
image_path = "images.png"
generated_code = generate_code(image_path, tokenizer, vit_model, decoder)
print("Generated code:\n", generated_code)
